In [ ]:
import nltk
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

STOPWORDS = set(stopwords.words('english'))
df = pd.read_csv('NFR_data.csv', encoding='latin1')

REPLACE_BY_SPACE = re.compile('[/(){}\|@,;]')
BAD_SYMBOLS = re.compile('[^0-9a-z #+_]')

ignore_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
    "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
    'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
    'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
    'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were',
    'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because',
    'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about',
    'against', 'between', 'into', 'through', 'during', 'before', 'after',
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there',
    'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few',
    'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only',
    'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will',
    'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm',
    'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
    'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
    "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't",
    '?', '%', '/', '(', ')', '[', ']', '-', ':', ';', 'system', 'product',
    'application', 'should', 'would', 'shall', 'go', 'System', 'system.',
    'System.','â€',"'",]

def clean_nfr(text):
    text = text.lower()
    text = REPLACE_BY_SPACE.sub(' ', text)
    text = BAD_SYMBOLS.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    text = ' '.join(word for word in text.split() if word not in ignore_words)
    return text

def tokenize_and_create_vocab(df, text_column):
    lemmatizer = WordNetLemmatizer()
    vocab = set()
    tokenized_texts = []
    for text in df[text_column]:
        cleaned_text = clean_nfr(text)
        tokens = word_tokenize(cleaned_text)
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        vocab.update(lemmatized_tokens)
        tokenized_texts.append(lemmatized_tokens)
    return tokenized_texts, vocab

def replace_oov(tokens, vocab):
    return [token if token in vocab else 'UKN' for token in tokens]

df['cleaned_sentence'] = df['sentence'].apply(clean_nfr)
tokenized_texts, vocab = tokenize_and_create_vocab(df, 'cleaned_sentence')
df['tokenized'] = tokenized_texts
df['tokenized_with_oov'] = df['tokenized'].apply(lambda tokens: replace_oov(tokens, vocab))
print(df["tokenized_with_oov"])

def word_embedding(df, text_column, max_words=5000, max_length=50):
    tokenizer = Tokenizer(num_words=max_words, oov_token='OOV')
    tokenizer.fit_on_texts(df[text_column])
    sequences = tokenizer.texts_to_sequences(df[text_column])
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
    print("Padded Sequences Shape:", padded_sequences.shape)
    return padded_sequences, tokenizer

padded_sequences, tokenizer = word_embedding(df, 'cleaned_sentence')
np.save('padded_sequences.npy', padded_sequences)

print("Padded sequences saved to 'padded_sequences.npy'")
print(df)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns

tfidv = TfidfVectorizer(max_features=5000, min_df=1, smooth_idf=True, norm='l2')
X = tfidv.fit_transform(df['sentence'].values)
Y = pd.get_dummies(df['class_name']).values

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=34, stratify=Y
)

def create_model(input_dim):
    model = Sequential([
        Dense(2128, activation="relu", input_dim=input_dim),
        Dropout(0.3),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(128, activation="relu"),
        Dropout(0.3),
        Dense(5, activation="softmax")
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model = create_model(X.shape[1])
history = model.fit(
    X_train, Y_train,
    epochs=5, batch_size=32,
    validation_data=(X_test, Y_test),
    callbacks=[reduce_lr, early_stop], verbose=2
)

def plot_history(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_history(history)

y_train_pred = model.predict(X_train)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true = np.argmax(Y_train, axis=1)
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
plt.show()

target_names = df['class_name'].unique()
print("Classification Report (Training Set):")
print(classification_report(y_train_true, y_train_pred_classes, target_names=target_names))
print(classification_report(y_true, y_pred_classes, target_names=target_names))

train_acc = model.evaluate(X_train, Y_train, verbose=False)[1]
test_acc = model.evaluate(X_test, Y_test, verbose=False)[1]
print(f"Training Accuracy: {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")
